In [1]:
import json
import re
from _pipeline import create_payload, model_req  # Import AI model interaction functions

# Load previous best prompts (Reinforcement Learning Feedback)
try:
    with open("best_prompts.json", "r") as file:
        best_prompt_history = json.load(file)  # Load previous best prompts and scores
except FileNotFoundError:
    best_prompt_history = {}

#### (1) AI Generates a Structured Prompt for Requirement Analysis (Level-1 Automation)
meta_prompt = (
    "Create a highly structured prompt that extracts detailed functional and non-functional "
    "requirements for an AI-powered Study Companion Discord Bot. The structured prompt should ensure "
    "clarity, completeness, and depth in the generated response."
)

payload_meta = create_payload(
    target="ollama",
    model="llama3.2:latest",
    prompt=meta_prompt,
    temperature=0.7  # Lower temperature for more deterministic output
)
_, generated_prompt = model_req(payload=payload_meta)

#### (2) AI Executes the Generated Prompt (Level-1 Execution)
payload_exec = create_payload(
    target="ollama",
    model="llama3.2:latest",
    prompt=generated_prompt,
    temperature=1.0,  # Higher creativity for detailed response
    num_ctx=4096,
    num_predict=2048
)
_, initial_response = model_req(payload=payload_exec)

#### (3) AI Evaluates & Suggests Improvements (Level-2 Automation)
eval_prompt = (
    f"Analyze the following requirement analysis response and suggest improvements for better clarity, "
    f"structure, or completeness:\n\n{initial_response}"
)

payload_eval = create_payload(
    target="ollama",
    model="llama3.2:latest",
    prompt=eval_prompt,
    temperature=0.7
)
_, improvement_suggestions = model_req(payload=payload_eval)

#### (4) AI Generates an Improved Prompt Based on Self-Evaluation
improve_prompt = (
    f"Revise the following prompt to incorporate the suggested improvements:\n\nGenerated Prompt: {generated_prompt}\n\n"
    f"Suggested Improvements: {improvement_suggestions}"
)

payload_improve = create_payload(
    target="ollama",
    model="llama3.2:latest",
    prompt=improve_prompt,
    temperature=0.7
)
_, improved_prompt = model_req(payload=payload_improve)

#### (5) AI Executes the Improved Prompt and Compares Responses
payload_final_exec = create_payload(
    target="ollama",
    model="llama3.2:latest",
    prompt=improved_prompt,
    temperature=1.0,
    num_ctx=4096,
    num_predict=2048
)
_, final_response = model_req(payload=payload_final_exec)

# Compare initial and final responses based on word count and keyword presence
important_terms = [
    "functional requirements", "non-functional requirements", "real-time study support",
    "personalized tutoring", "exam preparation"
]

def score_response(response):
    score = len(response.split())
    score += sum(1 for term in important_terms if term in response.lower()) * 10
    return score

initial_score = score_response(initial_response)
final_score = score_response(final_response)

best_prompt = improved_prompt if final_score > initial_score else generated_prompt
best_response = final_response if final_score > initial_score else initial_response
best_score = max(initial_score, final_score)

#### (6) Save the Best Performing Prompt in History
best_prompt_history[best_prompt] = {"feedback_score": best_score}

with open("best_prompts.json", "w") as file:
    json.dump(best_prompt_history, file, indent=4)

# Output the Best Optimized Prompt and Its AI-Generated Response
print(f"✅ Best Optimized Prompt: {best_prompt}")
print(f"📜 Response: {best_response}")
print(f"🏆 Score: {best_score}")


{'model': 'llama3.2:latest', 'prompt': 'Create a highly structured prompt that extracts detailed functional and non-functional requirements for an AI-powered Study Companion Discord Bot. The structured prompt should ensure clarity, completeness, and depth in the generated response.', 'stream': False, 'options': {'temperature': 0.7}}
{'model': 'llama3.2:latest', 'prompt': '**Prompt Structure:**\n\nTo generate a comprehensive set of functional and non-functional requirements for the AI-powered Study Companion Discord Bot, please provide the following information:\n\nI. **Introduction and Context**\n\n1. Briefly describe the purpose and scope of the Study Companion Discord Bot.\n2. Identify the target audience (e.g., students, educators, or both).\n3. Provide context on how the bot will be used to support learning and studying.\n\nII. **Functional Requirements**\n\n**A. User Interaction**\n\n1. Describe the types of user interactions that should be supported by the bot (e.g., text-based c